<h2> Work on Topic Modeling </h2>
<h3> Authors: CHOI Iatin, KAO Weiting, EL KHALIL Marwan, KHIEU Thomas, MAHON Matthew </h3>
<p> For this NLP work, we used the dataset provided for the exercice. The raw dataset is totally unexploitable and we add to first use VBA to make it readable for Python.</p>

#Excel VBA 
<br>
Sub AppendToExistingOnLeft()
<br>
Dim c As Range
<br>
For Each c In Selection
<br>
If c.Value <> "" Then c.Value = "https://" & c.Value
<br>
Next
<br>
End Sub

Once in a usable state we ran the code which is available at this address: https://nbviewer.jupyter.org/github/ThomasKhieu/Python/blob/master/NLPscraping.ipynb
<br>
This code was used to scrap the 70k rows of URLs in the file.

This second proposition makes a better use of machine learning than the first one while running less code iteration for the passes.

In [1]:
#import data
import pandas as pd
data = pd.read_csv('output.csv', error_bad_lines=False);
data_text = data[['Page Title']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
#quick look at the data
print(len(documents))
print(documents[:5])

146450
                                          Page Title  index
0  \n            Pôle-Emploi : les différentes ca...      0
1                                        Catégorie A      1
2                   \n            Newsletter              2
3  Ne pas déclarer ses revenus à pôle emploi [Rés...      3
4                                         2 réponses      4


In [3]:
#nltk and gensim import
import gensim
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(8888)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Phoenix/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#stemmer setup
stemmer = SnowballStemmer("french")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 4:
            result.append(lemmatize_stemming(token))
    return result

In [5]:
#postprocessing document preview
from nltk.stem.porter import *
doc_sample = documents[documents['index'] == 888].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Candy', 'Crush', 'Saga', ':', 'Trucs,', 'astuces', 'et', 'solutions']


 tokenized and lemmatized document: 
['candy', 'crush', 'truc', 'astuc', 'solut']


In [6]:
#saving processed results
documents = documents.dropna(subset=['Page Title'])
processed_docs = documents['Page Title'].map(preprocess)
processed_docs[:10]

0      [emploi, différent, catégor, chômeur]
1                                  [catégor]
2                                  [newslet]
3    [déclar, revenus, emploi, résolu, ferm]
4                                   [répons]
5                                  [newslet]
6                  [mont, plafond, ressourc]
7                                  [combien]
8                          [compl, ressourc]
9                    [alloc, rentr, scolair]
Name: Page Title, dtype: object

In [7]:
#bag-of-words
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 catégor
1 chômeur
2 différent
3 emploi
4 newslet
5 déclar
6 ferm
7 revenus
8 résolu
9 répons
10 mont


In [8]:
#datafilter
dictionary.filter_extremes(no_below=15, no_above=0.3, keep_n=100000)

We check the frequency of the words in our dataset to figure out if our data is cleaned and relevant or not.

In [9]:
from nltk import FreqDist
all_words = dictionary.iteritems()
fdist = FreqDist(all_words)
k = 100
top_k_words, _ = zip(*fdist.most_common(k))
print(top_k_words)

((0, 'catégor'), (1, 'différent'), (2, 'emploi'), (3, 'newslet'), (4, 'déclar'), (5, 'ferm'), (6, 'revenus'), (7, 'résolu'), (8, 'répons'), (9, 'mont'), (10, 'plafond'), (11, 'ressourc'), (12, 'combien'), (13, 'compl'), (14, 'alloc'), (15, 'rentr'), (16, 'scolair'), (17, 'condit'), (18, 'avoir'), (19, 'campagn'), (20, 'européen'), (21, 'foll'), (22, 'quitt'), (23, 'royaum'), (24, 'annonc'), (25, 'devr'), (26, 'démiss'), (27, 'theres'), (28, 'vendred'), (29, 'brut'), (30, 'éditorial'), (31, 'dessert'), (32, 'maison'), (33, 'cuisin'), (34, 'meilleur'), (35, 'recet'), (36, 'sabl'), (37, 'bordeau'), (38, 'meteo'), (39, 'jeud'), (40, 'matin'), (41, 'acced'), (42, 'apparaît'), (43, 'auss'), (44, 'billy'), (45, 'confirm'), (46, 'depuis'), (47, 'destin'), (48, 'didi'), (49, 'dériv'), (50, 'désign'), (51, 'désir'), (52, 'enfant'), (53, 'fêton'), (54, 'localis'), (55, 'masculin'), (56, 'mérovingien'), (57, 'national'), (58, 'prénom'), (59, 'rest'), (60, 'siecl'), (61, 'sixiem'), (62, 'souhait'),

In [10]:
my_stop_words = STOPWORDS.union(set(['linternaute','plateformes','plateforme','audio','semaine', 'heure','journal','newsletter','sommaire','programme','article', 'résolu','réponse','éditorial','combien','avoir','meilleur','différent', 'différente','différentes','moment','différents','réponses','fermé','devra','devrait','aussi','video','people','depuis','apparaît','fêtons','fêtont','confirmé','confirmés','assisté','assistée','notre','vidéo','alerte','never','viadeo','connect','demain','lundi','mardi','mercredi','jeudi','vendredi','samedi','dimanche','confirmer','newsletters','newsletter','programme','programmes','définitions','définition']))

In [11]:
#stemmer setup
stemmer = SnowballStemmer("french")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and token not in my_stop_words and len(token) > 4:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
#postprocessing document preview
from nltk.stem.porter import *
doc_sample = documents[documents['index'] == 888].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Candy', 'Crush', 'Saga', ':', 'Trucs,', 'astuces', 'et', 'solutions']


 tokenized and lemmatized document: 
['candy', 'crush', 'truc', 'astuc', 'solut']


In [13]:
#saving processed results
documents = documents.dropna(subset=['Page Title'])
processed_docs = documents['Page Title'].map(preprocess)

In [14]:
processed_docs

0                                [emploi, catégor, chômeur]
1                                                 [catégor]
2                                                        []
3                                 [déclar, revenus, emploi]
4                                                        []
5                                                        []
6                                 [mont, plafond, ressourc]
7                                                        []
8                                         [compl, ressourc]
9                                   [alloc, rentr, scolair]
10                                                 [condit]
11                                                       []
12           [royaum, foll, campagn, européen, croi, quitt]
13                                 [theres, annonc, démiss]
14                                           [brut, truand]
15                                        [dessert, maison]
16                                      

In [15]:
#bag-of-words
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 catégor
1 chômeur
2 emploi
3 déclar
4 revenus
5 mont
6 plafond
7 ressourc
8 compl
9 alloc
10 rentr


In [16]:
#datafilter
dictionary.filter_extremes(no_below=15, no_above=0.3, keep_n=100000)

In [17]:
from nltk import FreqDist
all_words = dictionary.iteritems()
fdist = FreqDist(all_words)
k = 100
top_k_words, _ = zip(*fdist.most_common(k))
print(top_k_words)

((0, 'catégor'), (1, 'emploi'), (2, 'déclar'), (3, 'revenus'), (4, 'mont'), (5, 'plafond'), (6, 'ressourc'), (7, 'compl'), (8, 'alloc'), (9, 'rentr'), (10, 'scolair'), (11, 'condit'), (12, 'campagn'), (13, 'européen'), (14, 'foll'), (15, 'quitt'), (16, 'royaum'), (17, 'annonc'), (18, 'démiss'), (19, 'theres'), (20, 'brut'), (21, 'dessert'), (22, 'maison'), (23, 'cuisin'), (24, 'meilleur'), (25, 'recet'), (26, 'sabl'), (27, 'bordeau'), (28, 'meteo'), (29, 'matin'), (30, 'acced'), (31, 'billy'), (32, 'confirm'), (33, 'destin'), (34, 'didi'), (35, 'dériv'), (36, 'désign'), (37, 'désir'), (38, 'enfant'), (39, 'localis'), (40, 'masculin'), (41, 'mérovingien'), (42, 'national'), (43, 'prénom'), (44, 'rest'), (45, 'siecl'), (46, 'sixiem'), (47, 'souhait'), (48, 'territoir'), (49, 'vérit'), (50, 'william'), (51, 'élanc'), (52, 'chien'), (53, 'horoscop'), (54, 'alic'), (55, 'never'), (56, 'canc'), (57, 'chinois'), (58, 'compt'), (59, 'factur'), (60, 'assist'), (61, 'aven'), (62, 'contrat'), (63

We take a random sample of our data.

In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[888]

[(47, 2), (747, 1)]

In [19]:
bow_doc_888 = bow_corpus[888]
for i in range(len(bow_doc_888)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_888[i][0], 
                                               dictionary[bow_doc_888[i][0]], 
bow_doc_888[i][1]))

Word 47 ("souhait") appears 2 time.
Word 747 ("conjugaison") appears 1 time.


We will use this random sample later on to check the strength of our LDA model.

In [20]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.8042160871156399), (1, 0.5943370131704818)]


In [21]:
#lda model training
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=100, workers=4)

In [22]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.040*"franc" + 0.035*"court" + 0.029*"coup" + 0.026*"biograph" + 0.025*"dat" + 0.022*"fill" + 0.022*"mond" + 0.021*"pierr" + 0.014*"coloriag" + 0.013*"citat"
Topic: 1 
Words: 0.074*"européen" + 0.064*"actual" + 0.062*"elect" + 0.049*"sondag" + 0.039*"résultat" + 0.036*"maman" + 0.022*"cann" + 0.019*"festival" + 0.017*"déclar" + 0.017*"surpris"
Topic: 2 
Words: 0.030*"recet" + 0.024*"météo" + 0.022*"cet" + 0.021*"voic" + 0.017*"meilleur" + 0.015*"franc" + 0.014*"travail" + 0.011*"condit" + 0.011*"comment" + 0.011*"indiqu"
Topic: 3 
Words: 0.190*"cuisin" + 0.038*"comment" + 0.035*"fair" + 0.013*"alic" + 0.012*"direct" + 0.012*"never" + 0.012*"astuc" + 0.011*"vrai" + 0.011*"frais" + 0.011*"quand"
Topic: 4 
Words: 0.046*"beaut" + 0.042*"photos" + 0.033*"nouvel" + 0.024*"cors" + 0.023*"nouveau" + 0.016*"dévoil" + 0.015*"résultat" + 0.015*"saint" + 0.014*"étoil" + 0.013*"vill"
Topic: 5 
Words: 0.062*"assist" + 0.032*"votr" + 0.020*"paris" + 0.017*"autr" + 0.016*"pass" + 0.0

In [23]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=100, workers=4)
lda_model_tfidf.save('model100.gensim')
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.016*"mont" + 0.013*"regl" + 0.013*"saint" + 0.012*"pass" + 0.012*"calcul" + 0.011*"verb" + 0.010*"droit" + 0.010*"comment" + 0.010*"frais" + 0.009*"salon"
Topic: 1 Word: 0.025*"court" + 0.021*"votr" + 0.019*"macron" + 0.018*"biograph" + 0.016*"pierr" + 0.015*"égal" + 0.013*"dat" + 0.012*"savoir" + 0.011*"condit" + 0.011*"emmanuel"
Topic: 2 Word: 0.133*"assist" + 0.038*"recet" + 0.022*"meilleur" + 0.015*"enfant" + 0.012*"figaro" + 0.012*"amour" + 0.011*"anniversair" + 0.011*"coloriag" + 0.011*"voisin" + 0.010*"petit"
Topic: 3 Word: 0.027*"hallyday" + 0.025*"johnny" + 0.018*"servic" + 0.015*"inform" + 0.015*"cann" + 0.013*"départ" + 0.012*"laetici" + 0.012*"festival" + 0.010*"nouveau" + 0.010*"saison"
Topic: 4 Word: 0.021*"voic" + 0.018*"lant" + 0.016*"comment" + 0.016*"lettr" + 0.014*"résum" + 0.012*"model" + 0.012*"maison" + 0.012*"lign" + 0.012*"carr" + 0.010*"demand"
Topic: 5 Word: 0.053*"européen" + 0.050*"actual" + 0.047*"elect" + 0.041*"sondag" + 0.033*"résultat" 

In [24]:
processed_docs[888]

['candy', 'crush', 'truc', 'astuc', 'solut']

In [25]:
for index, score in sorted(lda_model[bow_corpus[888]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7749892473220825	 
Topic: 0.038*"prénom" + 0.025*"lant" + 0.019*"départ" + 0.018*"donatien" + 0.018*"comm" + 0.018*"don" + 0.016*"enfant" + 0.016*"regl" + 0.016*"était" + 0.016*"siecl"

Score: 0.025004055351018906	 
Topic: 0.024*"femm" + 0.024*"apres" + 0.023*"toujour" + 0.022*"lettr" + 0.022*"inspir" + 0.022*"terr" + 0.017*"model" + 0.016*"bel" + 0.016*"grev" + 0.014*"carr"

Score: 0.025000834837555885	 
Topic: 0.040*"franc" + 0.035*"court" + 0.029*"coup" + 0.026*"biograph" + 0.025*"dat" + 0.022*"fill" + 0.022*"mond" + 0.021*"pierr" + 0.014*"coloriag" + 0.013*"citat"

Score: 0.025000834837555885	 
Topic: 0.074*"européen" + 0.064*"actual" + 0.062*"elect" + 0.049*"sondag" + 0.039*"résultat" + 0.036*"maman" + 0.022*"cann" + 0.019*"festival" + 0.017*"déclar" + 0.017*"surpris"

Score: 0.025000834837555885	 
Topic: 0.030*"recet" + 0.024*"météo" + 0.022*"cet" + 0.021*"voic" + 0.017*"meilleur" + 0.015*"franc" + 0.014*"travail" + 0.011*"condit" + 0.011*"comment" + 0.011*"indiqu"

Sco

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

In [26]:
for index, score in sorted(lda_model_tfidf[bow_corpus[888]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.43440860509872437	 
Topic: 0.133*"assist" + 0.038*"recet" + 0.022*"meilleur" + 0.015*"enfant" + 0.012*"figaro" + 0.012*"amour" + 0.011*"anniversair" + 0.011*"coloriag" + 0.011*"voisin" + 0.010*"petit"

Score: 0.36555996537208557	 
Topic: 0.016*"mont" + 0.013*"regl" + 0.013*"saint" + 0.012*"pass" + 0.012*"calcul" + 0.011*"verb" + 0.010*"droit" + 0.010*"comment" + 0.010*"frais" + 0.009*"salon"

Score: 0.025011049583554268	 
Topic: 0.020*"meteo" + 0.019*"franc" + 0.017*"grev" + 0.017*"guerr" + 0.015*"ultim" + 0.015*"horoscop" + 0.014*"mond" + 0.012*"premi" + 0.012*"blanc" + 0.011*"douleur"

Score: 0.025002911686897278	 
Topic: 0.025*"court" + 0.021*"votr" + 0.019*"macron" + 0.018*"biograph" + 0.016*"pierr" + 0.015*"égal" + 0.013*"dat" + 0.012*"savoir" + 0.011*"condit" + 0.011*"emmanuel"

Score: 0.025002911686897278	 
Topic: 0.027*"hallyday" + 0.025*"johnny" + 0.018*"servic" + 0.015*"inform" + 0.015*"cann" + 0.013*"départ" + 0.012*"laetici" + 0.012*"festival" + 0.010*"nouveau" + 

Same here.

In [29]:
#Testing model on unseen document
unseen_document = 'Meghan, épouse du prince Harry'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.8199965357780457	 Topic: 0.101*"sant" + 0.021*"franc" + 0.019*"premi" + 0.017*"symptôm" + 0.016*"meghan"
Score: 0.020000381395220757	 Topic: 0.040*"franc" + 0.035*"court" + 0.029*"coup" + 0.026*"biograph" + 0.025*"dat"
Score: 0.020000381395220757	 Topic: 0.074*"européen" + 0.064*"actual" + 0.062*"elect" + 0.049*"sondag" + 0.039*"résultat"
Score: 0.020000381395220757	 Topic: 0.030*"recet" + 0.024*"météo" + 0.022*"cet" + 0.021*"voic" + 0.017*"meilleur"
Score: 0.020000381395220757	 Topic: 0.190*"cuisin" + 0.038*"comment" + 0.035*"fair" + 0.013*"alic" + 0.012*"direct"
Score: 0.020000381395220757	 Topic: 0.046*"beaut" + 0.042*"photos" + 0.033*"nouvel" + 0.024*"cors" + 0.023*"nouveau"
Score: 0.020000381395220757	 Topic: 0.062*"assist" + 0.032*"votr" + 0.020*"paris" + 0.017*"autr" + 0.016*"pass"
Score: 0.020000381395220757	 Topic: 0.069*"conseil" + 0.066*"rédact" + 0.042*"hallyday" + 0.039*"johnny" + 0.019*"laetici"
Score: 0.020000381395220757	 Topic: 0.024*"femm" + 0.024*"apres" + 0

We can see that on an unknown dataset, the model works by giving a result related to Meghan.

In [28]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis
lda100 = gensim.models.ldamodel.LdaModel.load('model100.gensim')
lda_display100 = pyLDAvis.gensim.prepare(lda100, bow_corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display100)

/Users/Phoenix/spaCy2/.env/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


As part of our comparison:
The visualization allows us to see the term frequency for each topic. From this we have from our dataset the following insight for topic modeling:
<br>
<br>
Topic 1: Cooking advice
<br>
Topic 2: Unknown (because of stemming)
<br>
Topic 3: Macron biography
<br>
Topic 4: Johnny Hallyday
<br>
Topic 5: How to write a cover letter
<br>
Topic 6: European elections poll
<br>
Topic 7: Chef cooking
<br>
Topic 8: Redaction advice
<br>
Topic 9: Weather in France
<br>
Topic 10: Healthcare
<br>
<br>

We can see here that the result are slightly better. However it is still far from being exploitable from a business perspective. We believe that most of it is due to the use of french language in our dataset which limits Gensim frozenset, we would need to exclude manually a lot of words.